In [1]:
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import urllib.request
import time
import bs4 as bs
from geopy.geocoders import Nominatim

import seaborn as sns
from scipy import stats

# To make pyproj conversions work
import os 
os.environ['PROJ_LIB']=r"C:\Users\spiris\AppData\Local\Continuum\anaconda3\Library\share"


In [2]:
total_pages = 500
urls = pd.DataFrame(columns=['url'])
urls_added = 0
page = 1

for page in range(1,(total_pages + 1)):    
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    url = "https://www.zoocasa.com/toronto-on-sold-listings?page=" + str(page)
    request = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    data_raw = urllib.request.urlopen(request).read()
    # split the house listing links
    data_split = data_raw.split(b'"id"')[3:27]
    time.sleep(1)
    
    for listing in range(24): 
        try:
            start = data_split[listing].find(b'path"')
            end = data_split[listing].find(b'","province')
            urls.loc[urls_added] = 'https://www.zoocasa.com/' + str(data_split[listing][(start+8):(end)])[2:-1]
            urls_added += 1
        except:
            urls_added += 0
        
urls.to_csv('house_urls.csv')


In [3]:
links = pd.read_csv('house_urls.csv', index_col='Unnamed: 0')
data = pd.DataFrame(columns=['title','final_price', 'list_price','bedrooms','partial_bedrooms','bathrooms',
                             'bathrooms_partial','sqft_min','sqft_max','parking','description','mls',
                             'type','full_link','full_address', 'list_date'])

In [4]:
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,} 


#read data, convert to 2 formats
request=urllib.request.Request(url,None,headers) #The assembled request
data_raw = urllib.request.urlopen(request).read()
soup = bs.BeautifulSoup(data_raw, features="html")

In [37]:
# for unit testing, check if the scrapper for work the first listing
request=urllib.request.Request(urls.url[2],None,headers) #The assembled request
data_raw = urllib.request.urlopen(request).read()
soup = bs.BeautifulSoup(data_raw, "html")
data.loc[1,'full_link'] = url
data.loc[1,'title'] = soup.title.contents[0]
mls = data_raw.find(b'mlsNum')
data.loc[1,'mls'] = str(data_raw[(mls+9):((mls+17))])[2:-1]

#Home Type - completed
type_end = data_raw.find(b',"levels"')
type_start = data_raw.find(b'"type":"',type_end - 30,type_end)
data.loc[1,'type'] = str(data_raw[(type_start+8):((type_end-1))])[2:-1]


#list Price - completed
list_Price_end = data_raw.find(b'"soldPrice"')
list_Price_start = data_raw.find(b'"price"',list_Price_end - 30,list_Price_end)
data.loc[1,'list_price'] = int(str(data_raw[(list_Price_start+8):((list_Price_end-1))])[2:-1])

#final Price - completed
final_Price_end = data_raw.find(b'"bedrooms"')
final_Price_start = data_raw.find(b'"soldPrice"',final_Price_end - 30,final_Price_end)
try:
    data.loc[1,'final_price'] = int(str(data_raw[(final_Price_start+12):((final_Price_end-1))])[2:-1])
except:
    data.loc[1,'final_price'] = 0
    

try:
    for span in soup.find_all('span',class_ = "style_value__SVQPi undefined"):
        date = span.text
        break
    data.loc[1, 'list_date'] = date
except:
    data.loc[1, 'list_date'] = 0

    
    # for bedroom & bathroom information, we can use soup's class identifier
#house_info = soup.find_all(class_ = "_1cRMCD7z _1WkjEOaf")
#try:
#    data.loc[1, 'bedrooms'] = int(house_info.find(class_="_38gj6pIW icon-bedroom").get_text())
#except:
#    data.loc[1, 'bedrooms'] = 0
#bathrooms = int(house_info.find(class_="_38gj6pIW icon-bathroom").get_text())

In [5]:
data

,title,final_price,list_price,bedrooms,partial_bedrooms,bathrooms,bathrooms_partial,sqft_min,sqft_max,parking,description,mls,type,full_link,full_address,list_date


In [85]:
#data.head(1)
#house_info

#soup = bs.BeautifulSoup(data_raw, features="lxml")
#soup = bs.BeautifulSoup(data_raw, 'html.parser')

import requests
  headers = {'User-Agent': 'Chrome/39.0.2171.95'}

from splinter import Browser
executable_path = {'executable_path':'C:\\Users\\Zack Pan\\Downloads\\chromedriver.exe'}
browser = Browser('chrome', **executable_path)

url = 'https://www.zoocasa.com/toronto-on-real-estate/38-victor-ave'
#url = 'https://www.zoocasa.com/toronto-on-real-estate/riverdale?page=1&listing-id=10744774'

#response = requests.get(url, headers)

#soup = bs.BeautifulSoup(response.text, 'html.parser')
#for div in soup.find_all('div', class_='style_grid__1WWi7 style_row__2A8BY style_active__1aXT0 false'):
#for div in soup.find_all('span', class_='style_component__2LlMv undefined style_status-label__3_pmx'):
    #print(div.text)
    #date = span.text
    #break

#87d9kyj!TziPKkJ


In [97]:
url = 'https://www.zoocasa.com/toronto-on-real-estate/riverdale?page=1&listing-id=10744774'
browser.visit(url)
browser.execute_script("window.scrollTo(200, document.body.scrollHeight);")

print(browser.find_by_css('div').text)

Sold Prices
Calculator
Map
Market Insights
My Searches
Zack
Menu
CanadaOntarioTorontoRiverdale
Listings
Buildings
Provinces
Search
Buy
Active Listings
More
Map
Real Estate
Price (Low To High)
Price (High To Low)
Newest To Oldest
Oldest To Newest
Bedrooms (High To Low)
Bedrooms (Low To High)
Bathrooms (High To Low)
Bathrooms (Low To High)
For Sale
$999,999
Added 2 Days Ago
42 Wroxeter Ave
3 Bed
3 Bath
1500–2000 sqft
For Sale
$1,499,900
Added 3 Days Ago
135 Langley Ave
3 Bed
3 Bath
N/A sqft
For Sale
$989,000
Added 3 Days Ago
45 Blake St
3 Bed
4 Bath
N/A sqft
For Sale
$1,699,000
Added 3 Days Ago
1 Lydia Crt
5 Bed
4 Bath
2500–3000 sqft
For Sale
$1,349,000
Added 4 Days Ago
122 Ivy Ave
3 Bed
3 Bath
1500–2000 sqft
For Sale
$1,699,000
Added 5 Days Ago
72 Harcourt Ave
3 Bed
3 Bath
N/A sqft
For Sale
$1,299,000
Added 5 Days Ago
116 Galt Ave
3 Bed
3 Bath
N/A sqft
For Sale
$1,089,900
Added 5 Days Ago
59 Bloomfield Ave
3 Bed
2 Bath
N/A sqft
For Sale
$1,099,900
Added 6 Days Ago
118 Wolfrey Ave
3 Bed


In [6]:
house_num = links.shape[0]
url_error_tracker = []

for i in range(house_num):
    try:
        request=urllib.request.Request(urls.url[i],None,headers) #The assembled request
        data_raw = urllib.request.urlopen(request).read()
        soup = bs.BeautifulSoup(data_raw, features="html")

        #full link
        data.loc[i,'full_link'] = url

        #title - completed
        data.loc[i,'title'] = soup.title.contents[0]

        #MLS - multiple listing number completed
        mls = data_raw.find(b'mlsNum')
        data.loc[i,'mls'] = str(data_raw[(mls+9):((mls+17))])[2:-1]

        #Home Type - completed
        type_end = data_raw.find(b',"levels"')
        type_start = data_raw.find(b'"type":"',type_end - 30,type_end)
        data.loc[i,'type'] = str(data_raw[(type_start+8):((type_end-1))])[2:-1]


        #list Price - completed
        list_Price_end = data_raw.find(b'"soldPrice"')
        list_Price_start = data_raw.find(b'"price"',list_Price_end - 30,list_Price_end)
        data.loc[i,'list_price'] = int(str(data_raw[(list_Price_start+8):((list_Price_end-1))])[2:-1])

        #final Price - completed
        final_Price_end = data_raw.find(b'"bedrooms"')
        final_Price_start = data_raw.find(b'"soldPrice"',final_Price_end - 30,final_Price_end)
        try:
            data.loc[i,'final_price'] = int(str(data_raw[(final_Price_start+12):((final_Price_end-1))])[2:-1])
        except:
            data.loc[i,'final_price'] = 0

        '''
        # for bedroom & bathroom information, we can use soup's class identifier
        house_info = soup.find_all(class_ = "_1cRMCD7z _1WkjEOaf")[0]
        try:
            data.loc[i, 'bedrooms'] = int(house_info.find(class_="_38gj6pIW icon-bedroom").get_text())
        except:
            data.loc[i, 'bedrooms'] = 0
        bathrooms = int(house_info.find(class_="_38gj6pIW icon-bathroom").get_text())
        #sqft = house_info.find(class_="_38gj6pIW icon-house").get_text()

        '''
        
        # bedrooms - completed
        bedrooms_end = data_raw.find(b'"bedroomsPartial"')
        bedrooms_start = data_raw.find(b'"bedrooms"',bedrooms_end - 30,bedrooms_end)
        try:
            data.loc[i, 'bedrooms'] = int(str(data_raw[(bedrooms_start+11):((bedrooms_end-1))])[2:-1])
        except:
            data.loc[i, 'bedrooms'] = 0
        


        # partial bedrooms - completed
        partial_bedrooms_end = data_raw.find(b'"bathrooms"')
        partial_bedrooms_start = data_raw.find(b'"bedroomsPartial"',partial_bedrooms_end - 30,partial_bedrooms_end)
        try:
            data.loc[i, 'partial_bedrooms'] = int(str(data_raw[(partial_bedrooms_start+18):((partial_bedrooms_end-1))])[2:-1])
        except:
            data.loc[i, 'partial_bedrooms'] = 0

        
        # bathrooms - completed
        bathrooms_end = data_raw.find(b'"bathroomsPartial"')
        bathrooms_start = data_raw.find(b'"bathrooms"',bathrooms_end - 30,bathrooms_end)
        try:
            data.loc[i, 'bathrooms'] = int(str(data_raw[(bathrooms_start+12):((bathrooms_end-1))])[2:-1])
        except:
            data.loc[i, 'bathrooms'] = 0
        
        '''
        # similar to bedroom, the bathrooms can also be found using the soup
        try:
            data.loc[i, 'bathrooms'] = int(house_info.find(class_="_38gj6pIW icon-bathroom").get_text())
        except:
            data.loc[i, 'bathrooms'] = 0
        '''

        # bathrooms partial - Completed
        bathrooms_partial_end = data_raw.find(b'"squareFootage"')
        bathrooms_partial_start = data_raw.find(b'"bathroomsPartial"',bathrooms_partial_end - 30,bathrooms_partial_end)
        try:
            data.loc[i, 'bathrooms_partial'] = int(str(data_raw[(bathrooms_partial_start+18):((bathrooms_partial_end-1))])[2:-1])
        except:
            data.loc[i, 'bathrooms_partial'] = 0


        # squared footage minimum
        sqft_min_end = data_raw.find(b'"max"')
        sqft_min_start = data_raw.find(b'"min"',sqft_min_end - 30,sqft_min_end)
        try:
            data.loc[i, 'sqft_min'] = int(str(data_raw[(sqft_min_start+6):((sqft_min_end-1))])[2:-1])
        except:
            data.loc[i, 'sqft_min'] = 0   


        # squared footage maximum - completed
        sqft_max_end = data_raw.find(b'"isVow"')
        sqft_max_start = data_raw.find(b'"max"',sqft_max_end - 30,sqft_max_end)
        try:
            data.loc[i, 'sqft_max'] = int(str(data_raw[(sqft_max_start+6):((sqft_max_end-2))])[2:-1])
        except:
            data.loc[i, 'sqft_max'] = 0

        # parking - completed
        parking_end = data_raw.find(b'"maintenanceFees"')
        parking_start = data_raw.find(b'"parking"',parking_end - 30,parking_end)
        try:
            data.loc[i, 'parking'] = int(str(data_raw[(parking_start+10):((parking_end-1))])[2:-1])
        except: 
            data.loc[i, 'parking'] = 0

        #location data
        address = soup.title.contents[0]
        address = address.replace('Circ', 'Cir') #edit #1
        address_b = address.find('(')
        address_a = address.find('-')  

        data.loc[i,'full_address'] = address[(address_a+1):address_b]
        
    except:
        print('House information download error with url number: ', i)
        url_error_tracker.append(i)
        continue

In [7]:
data

,title,final_price,list_price,bedrooms,partial_bedrooms,bathrooms,bathrooms_partial,sqft_min,sqft_max,parking,description,mls,type,full_link,full_address,list_date
0,"77 Invermay Ave, Toronto (C5001119) | Zoocasa",2777000,2799900,4,1,6,0,0,0,6,NaN,C5001119,Detached,https://www.zoocasa.com/toronto-on-sold-listin...,"77 Invermay Ave, Toronto",NaN
1,"th32 - 80 Carr St, Toronto (C5222055) | Zoocasa",720000,599000,2,0,1,0,700,799,1,NaN,C5222055,Condo Townhouse,https://www.zoocasa.com/toronto-on-sold-listin...,"80 Carr St, Toronto",NaN
2,"712 Balliol St, Toronto (C5219621) | Zoocasa",1651000,1449000,3,0,2,0,1100,1500,2,NaN,C5219621,Att/Row/Twnhouse,https://www.zoocasa.com/toronto-on-sold-listin...,"712 Balliol St, Toronto",NaN
3,"702 - 1470 Midland Ave, Toronto (E5219142) | Z...",590000,549000,2,0,2,0,1200,1399,1,NaN,E5219142,Condo Apt,https://www.zoocasa.com/toronto-on-sold-listin...,"1470 Midland Ave, Toronto",NaN
4,"324 - 364 The East Mall St, Toronto (W5191950)...",537000,539000,3,0,3,0,1200,1399,1,NaN,W5191950,Condo Townhouse,https://www.zoocasa.com/toronto-on-sold-listin...,"364 The East Mall St, Toronto",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,"927 - 36 Blue Jays Way, Toronto (C5218441) | Z...",885000,799999,2,0,2,0,900,999,1,NaN,C5218441,Condo Apt,https://www.zoocasa.com/toronto-on-sold-listin...,"36 Blue Jays Way, Toronto",NaN
11996,"1601 - 197 Yonge St, Toronto (C5217786) | Zoocasa",680000,599990,1,1,1,0,500,599,0,NaN,C5217786,Condo Apt,https://www.zoocasa.com/toronto-on-sold-listin...,"197 Yonge St, Toronto",NaN
11997,"9e - 8 Rosebank Dr, Toronto (E5217749) | Zoocasa",558000,561900,2,0,1,0,800,899,1,NaN,E5217749,Condo Apt,https://www.zoocasa.com/toronto-on-sold-listin...,"8 Rosebank Dr, Toronto",NaN
11998,"719 - 20 Scrivener Sq, Toronto (C5217608) | Zo...",1071000,989000,2,0,2,0,800,899,1,NaN,C5217608,Comm Element Condo,https://www.zoocasa.com/toronto-on-sold-listin...,"20 Scrivener Sq, Toronto",NaN


In [8]:
data.to_csv('house_details.csv')

In [ ]:
for x in range(len(data)):
    try:
        time.sleep(1) #to add delay in case of large DFs
        geocode_result = gmaps.geocode(data['full_address'][x])
        data['lat'][x] = geocode_result[0]['geometry']['location'] ['lat']
        data['long'][x] = geocode_result[0]['geometry']['location']['lng']
    except IndexError:
        print("Address was wrong...")
    except Exception as e:
        print("Unexpected error occurred.", e )
data.head()

In [ ]:
data.to_csv('house_details.csv')

In [4]:
#get Toronto neighbourhood data and extract district codes with their average incomes
t_data = pd.read_csv('datasets/housing/toronto_cenus_data_2016.csv' )
# Potential Income prospects, ids: 1030, 2273, 2274

t_income = t_data[(t_data.Characteristic == 'Neighbourhood Number') 
                  | (t_data.Characteristic == 'Total income: Average amount ($)')]
t_income = t_income.T.reset_index()
t_income.columns = ['area', 'area_code', 'average_income']
t_income = t_income.iloc[6:,:]

t_income['average_income'] = t_income['average_income'].apply(lambda a: a.replace(',','')).astype('int64')
t_income['area_code'] = t_income['area_code'].astype('int64')
print(t_income.info())
print('-' * 60)

# Calculate income mean
mean_t_income = t_income['average_income'].mean()
print('GTA mean income: ',mean_t_income)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 6 to 145
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   area            140 non-null    object
 1   area_code       140 non-null    int64 
 2   average_income  140 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ KB
None
------------------------------------------------------------
GTA mean income:  55248.49285714285


In [9]:
t_data.head()

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,...,37,7,137,64,60,94,100,97,27,31
1,2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,...,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526",...,"16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
3,4,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177",...,"15,004","21,343","53,350","11,703","7,826","13,986","10,578","11,652","27,713","14,687"
4,5,Population,Population and dwellings,Census Profile 98-316-X2016001,Population Change 2011-2016,4.50%,-3.90%,8.00%,1.30%,4.60%,...,12.90%,3.80%,0.30%,7.20%,0.50%,2.60%,11.70%,7.50%,-0.40%,0.80%


In [30]:
data = pd.read_csv('house_details.csv')

In [7]:
data

,Unnamed: 0,title,final_price,list_price,bedrooms,partial_bedrooms,bathrooms,bathrooms_partial,sqft_min,sqft_max,parking,description,mls,type,full_link,full_address,lat,long,city_district,neighbourhood
0,0,"27 Flanders Rd, Toronto (C4943888) | Zoocasa",2800000,2699000,4,1,4,0,0,0,3,NaN,C4943888,Detached,https://www.zoocasa.com/toronto-on-sold-listin...,"27 Flanders Rd, Toronto",43.698641,-79.433357,NaN,NaN
1,1,"133 Brookdale Ave, Toronto (C4941663) | Zoocasa",1640000,1659000,3,0,2,0,0,0,1,NaN,C4941663,Detached,https://www.zoocasa.com/toronto-on-sold-listin...,"133 Brookdale Ave, Toronto",43.728845,-79.407935,NaN,NaN
2,2,"toronto - 329 Woodfield Rd, Toronto (E4941801)...",1265000,999000,3,0,2,0,0,0,1,NaN,E4941801,Semi-Detached,https://www.zoocasa.com/toronto-on-sold-listin...,"329 Woodfield Rd, Toronto",43.673144,-79.324276,NaN,NaN
3,3,"107 Winchester St, Toronto (C4940794) | Zoocasa",1875000,1650000,4,0,2,0,2000,2500,0,NaN,C4940794,Att/Row/Twnhouse,https://www.zoocasa.com/toronto-on-sold-listin...,"107 Winchester St, Toronto",43.666409,-79.365269,NaN,NaN
4,4,"1614 - 710 Humberwood Blvd, Toronto (W4939984)...",572000,575000,2,1,2,0,900,999,1,NaN,W4939984,Comm Element Condo,https://www.zoocasa.com/toronto-on-sold-listin...,"710 Humberwood Blvd, Toronto",43.724480,-79.616170,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,115,"26 Kings Park Blvd, Toronto (E4935203) | Zoocasa",1350000,1099000,4,1,2,0,0,0,1,NaN,E4935203,Semi-Detached,https://www.zoocasa.com/toronto-on-sold-listin...,"26 Kings Park Blvd, Toronto",43.685626,-79.345610,NaN,NaN
116,116,"518 - 21 Clairtrell Rd, Toronto (C4934860) | Z...",695000,649900,2,0,2,0,800,899,1,NaN,C4934860,Condo Apt,https://www.zoocasa.com/toronto-on-sold-listin...,"21 Clairtrell Rd, Toronto",43.768151,-79.390791,NaN,NaN
117,117,"158 Fulton Ave, Toronto (E4934821) | Zoocasa",1350000,1399000,3,0,2,0,0,0,0,NaN,E4934821,Semi-Detached,https://www.zoocasa.com/toronto-on-sold-listin...,"158 Fulton Ave, Toronto",43.683391,-79.351099,NaN,NaN
118,118,"9 - 988 Sheppard Ave W, Toronto (C4934099) | Z...",785000,749800,3,0,3,0,1400,1599,2,NaN,C4934099,Condo Townhouse,https://www.zoocasa.com/toronto-on-sold-listin...,"988 Sheppard Ave W, Toronto",43.751433,-79.459771,NaN,NaN
